In [19]:
def parse_log(log_path: str) -> (list[dict], list[dict]):
    subs: list[dict] = []
    timers: list[dict] = []
    base_wall_time = 0
    base_ros_time = 0
    timer_trigger_flag = False

    with open(log_path, 'r') as f:
        for i, line in enumerate(f.readlines()):
            split = line.split(' ')
            if i == 0:
                base_wall_time = int(split[0])
                base_ros_time = int(split[1])

            if timer_trigger_flag:
                if 'Received message ID:' not in line:
                    timer_trigger_flag = False
                else:
                    if timers[-1].get('received_subs'):
                        timers[-1]['received_subs'].append(int(split[-1]))
                    else:
                        timers[-1]['received_subs'] = [int(split[-1])]

            if '[TimerDependencyNode] Sub Start. ID:' in line:
                subs.append(
                    {'id': int(split[-1]),
                     'start_wall_time': int(split[0]) - base_wall_time,
                     'start_ros_time': int(split[1]) - base_ros_time})
            elif '[TimerDependencyNode] Sub End. ID:' in line:
                subs[-1].update(
                    [('end_wall_time', int(split[0]) - base_wall_time),
                     ('end_ros_time', int(split[1]) - base_ros_time)])
            elif '[TimerDependencyNode] Timer triggered. ID:' in line:
                timers.append(
                    {'id': int(split[-1]),
                     'start_wall_time': int(split[0]) - base_wall_time,
                     'start_ros_time': int(split[1]) - base_ros_time})
                timer_trigger_flag = True

    return subs, timers


In [20]:
from bokeh.plotting import figure, show
from bokeh.models import Arrow, OpenHead, NormalHead, VeeHead
from bokeh.io import output_notebook
output_notebook()


def get_ms(data: dict, start_or_end: str, wall_or_ros: str) -> float:
    if wall_or_ros == 'wall' and start_or_end == 'start':
        return data['start_wall_time'] * 10 ** (-6)
    elif wall_or_ros == 'wall' and start_or_end == 'end':
        return data['end_wall_time'] * 10 ** (-6)
    elif wall_or_ros == 'ros' and start_or_end == 'start':
        return data['start_ros_time'] * 10 ** (-6)
    elif wall_or_ros == 'ros' and start_or_end == 'end':
        return data['end_ros_time'] * 10 ** (-6)
    else:
        print('Invalid argument')


def plot_message_flow(subs, timers, wall_or_ros: str, title: str):
    p = figure(
        height=200, width=1200, title=title,
        x_axis_label='Wall Time [ms]' if wall_or_ros == 'wall' else 'ROS Time [ms]', y_range=(0, 0.3),
        x_range=(0, get_ms(subs[-1], 'end', wall_or_ros)),
        tools='xwheel_zoom,xpan,reset,save')
    p.grid.grid_line_color = None
    p.yaxis.major_label_text_font_size = '0pt'

    # Plot subs
    sub_end_id_x_map = {}
    for sub in subs:
        # start
        start_arrow = Arrow(
            end=VeeHead(size=6, fill_color="green", line_color="green"),
            line_color="green", x_start=get_ms(sub, 'start', wall_or_ros),
            y_start=0.2, x_end=get_ms(sub, 'start', wall_or_ros),
            y_end=0.3)
        p.add_layout(start_arrow)
        # end
        sub_end_id_x_map[sub['id']] = sub['end_wall_time'] * 10 ** (-6)
        end_arrow = Arrow(
            end=VeeHead(size=6, fill_color="red", line_color="red"),
            line_color="red", x_start=get_ms(sub, 'end', wall_or_ros),
            y_start=0.3, x_end=get_ms(sub, 'end', wall_or_ros),
            y_end=0.2)
        p.add_layout(end_arrow)

    # Plot timers
    for timer in timers:
        # start
        start_arrow = Arrow(
            end=VeeHead(size=6, fill_color="green", line_color="green"),
            line_color="green", x_start=get_ms(timer, 'start', wall_or_ros),
            y_start=0, x_end=get_ms(timer, 'start', wall_or_ros),
            y_end=0.1)
        p.add_layout(start_arrow)
        # received subs
        if timer.get('received_subs'):
            for sub_id in timer['received_subs']:
                sub = subs[sub_id]
                arrow = Arrow(
                    end=NormalHead(size=4, fill_color="gray", line_color="gray"),
                    line_color="gray", x_start=sub_end_id_x_map[sub['id']],
                    y_start=0.2, x_end=get_ms(timer, 'start', wall_or_ros),
                    y_end=0.1)
                p.add_layout(arrow)

    show(p)


Loading BokehJS ...

In [21]:
subs_1, timers_1 = parse_log('output/1.0_log.txt')


In [22]:
plot_message_flow(subs_1, timers_1, 'wall', '[1.0] Wall Time')
plot_message_flow(subs_1, timers_1, 'ros', '[1.0] ROS Time')


In [23]:
subs_0_5, timers_0_5 = parse_log('output/0.5_log.txt')


In [24]:
plot_message_flow(subs_0_5, timers_0_5, 'wall', '[0.5] Wall Time')
plot_message_flow(subs_0_5, timers_0_5, 'ros', '[0.5] ROS Time')
